In [1]:
import pandas as pd
import numpy as np
import sklearn
from xgboost import XGBClassifier
import xgboost
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import pickle
from sklearn.utils import shuffle

In [16]:
df = pd.read_csv('../data/train.csv', delimiter=",", usecols = ['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv', 'click_bool', 'gross_bookings_usd',
       'booking_bool'])
neededVars = ['click_bool','booking_bool']
df = shuffle(df)

Lets train it on clicking

In [17]:
trainOnlyColumns = ['position','click_bool','booking_bool','gross_bookings_usd']
Y = df['click_bool']
dfNeeded= df[neededVars]
df.drop(trainOnlyColumns, axis=1,inplace=True)


In [25]:
df.columns

Index(['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv'],
      dtype='object')

In [18]:
n = 20
numOfRowsTest = (int(len(df)*(n/100)))

#Test
X_test = df.tail(numOfRowsTest)
Y_test = Y[-numOfRowsTest:]
dfNeeded_test = dfNeeded.tail(numOfRowsTest)

#Train
dfNeeded = dfNeeded.drop(dfNeeded.tail(numOfRowsTest).index,inplace=True)
df.drop(df.tail(numOfRowsTest).index,inplace=True)
y_train = Y[:(len(Y)-numOfRowsTest)]

In [5]:
# construct xgboost.DMatrix from numpy array, treat -999.0 as missing value
#xgmat = xgboost.DMatrix( df, label=y_train)
xgmat = xgboost.DMatrix( df, label=y_train)
# setup parameters for xgboost
param = {}
# use logistic regression loss, use raw prediction before logistic transformation
# since we only need the rank
param['objective'] = 'reg:logistic'
# scale weight of positive examples
param['scale_pos_weight'] = sum(Y==0)/sum(Y==1)
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'
param['silent'] = 0
param['nthread'] = 10

# you can directly throw param in, though we want to watch multiple metrics here
plst = list(param.items())

watchlist = [ (xgmat,'train') ]
# boost 120 trees
num_round = 120
print ('loading data end, start to boost trees')
bst = xgboost.train( plst, xgmat, num_round, watchlist ,early_stopping_rounds=5);
# save out model
#bst.save_model('clickingExtra.model')

print ('finish training')

# fit model no training data
#model = XGBClassifier()
#model.fit(xs, ys)
# make predictions for test data

E:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
E:\Anaconda\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


loading data end, start to boost trees
[0]	train-auc:0.664749
Will train until train-auc hasn't improved in 5 rounds.
[1]	train-auc:0.667174
[2]	train-auc:0.669493
[3]	train-auc:0.671117
[4]	train-auc:0.672273
[5]	train-auc:0.673101
[6]	train-auc:0.674487
[7]	train-auc:0.675528
[8]	train-auc:0.676346
[9]	train-auc:0.677349
[10]	train-auc:0.677882
[11]	train-auc:0.678372
[12]	train-auc:0.679235
[13]	train-auc:0.679792
[14]	train-auc:0.680381
[15]	train-auc:0.681008
[16]	train-auc:0.681589
[17]	train-auc:0.682404
[18]	train-auc:0.682985
[19]	train-auc:0.683668
[20]	train-auc:0.683979
[21]	train-auc:0.684597
[22]	train-auc:0.685095
[23]	train-auc:0.6855
[24]	train-auc:0.686122
[25]	train-auc:0.686589
[26]	train-auc:0.687211
[27]	train-auc:0.68765
[28]	train-auc:0.688193
[29]	train-auc:0.688597
[30]	train-auc:0.68901
[31]	train-auc:0.689552
[32]	train-auc:0.690044
[33]	train-auc:0.690548
[34]	train-auc:0.691019
[35]	train-auc:0.6914
[36]	train-auc:0.691655
[37]	train-auc:0.69201
[38]	train

In [19]:
def getScore(df):
    df['rank_srch_id'] = df.groupby('srch_id').cumcount().add(1)
    df['gain'] = 5*df['booking_bool']
    df['gain'] = np.where(df['gain'] == 0, df['click_bool'], df['gain'])

    df['g/rank'] = df['gain']/df['rank_srch_id']
    gi_sum = df['g/rank'].sum()
    
    df['gain_sorted'] = list(df[['srch_id', 'booking_bool', 'click_bool', 'gain']].sort_values(by = ['srch_id','booking_bool','click_bool'], ascending = [True, False, False])['gain'])
    df['g/rank_sorted'] = df['gain_sorted']/df['rank_srch_id']
    gi_sorted_sum = df['g/rank_sorted'].sum() 
    score = gi_sum/gi_sorted_sum
    return score, gi_sum, gi_sorted_sum

In [20]:
xgmat_test = xgboost.DMatrix( X_test, label=Y_test)
y_pred = bst.predict(xgmat_test)
X_test = pd.concat([X_test,dfNeeded_test],axis=1)
X_test['y_pred'] = y_pred

In [21]:
score, gi_sum, gi_sorted_sum = getScore(X_test.sort_values(by=['srch_id', 'y_pred'], ascending = [True, False]).reset_index(drop=True))
print(score)

# evaluate predictions
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
Counter(predictions)

0.6728051159536493
Accuracy: 66.93%


Counter({0.0: 652763, 1.0: 338906})

Lets train it on booking

In [ ]:
trainOnlyColumns = ['position','click_bool','booking_bool','gross_bookings_usd']
Y = df['booking_bool']
df.drop(trainOnlyColumns, axis=1,inplace=True)
df.drop(['date'], axis=1,inplace=True)

In [9]:
n = 20
numOfRowsTest = (int(len(df)*(n/100)))
X_test = df.tail(numOfRowsTest)
Y_test = Y[-numOfRowsTest:]
df.drop(df.tail(numOfRowsTest).index,inplace=True)
y_train = Y[:(len(Y)-numOfRowsTest)]

In [ ]:
# construct xgboost.DMatrix from numpy array, treat -999.0 as missing value
xgmat = xgboost.DMatrix( df, label=Y)

# setup parameters for xgboost
param = {}
# use logistic regression loss, use raw prediction before logistic transformation
# since we only need the rank
param['objective'] = 'reg:logistic'
# scale weight of positive examples
param['scale_pos_weight'] = sum(y_train==0)/sum(y_train==1)
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'
param['silent'] = 0
param['nthread'] = 10

# you can directly throw param in, though we want to watch multiple metrics here
plst = list(param.items())+[('eval_metric', 'ams@0.15')]

watchlist = [ (xgmat,'train') ]
# boost 120 trees
num_round = 25
print ('loading data end, start to boost trees')
bst = xgboost.train( plst, xgmat, num_round, watchlist , early_stopping_rounds=5);
# save out model
bst.save_model('bookingModelExtra.model')

print ('finish training')

# fit model no training data
#model = XGBClassifier()
#model.fit(xs, ys)
# make predictions for test data

In [ ]:
xgmat_test = xgboost.DMatrix( X_test, label=Y_test)
y_pred = bst.predict(xgmat_test)
X_test = pd.concat([X_test,dfNeeded_test],axis=1)
X_test['y_pred'] = y_pred

In [ ]:
score, gi_sum, gi_sorted_sum = getScore(X_test.sort_values(by=['srch_id', 'y_pred'], ascending = [True, False]).reset_index(drop=True))
print(score)

# evaluate predictions
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
Counter(predictions)

Lets make a submission

In [7]:
bst = xgboost.Booster({'nthread': 12})  # init model
bst.load_model('xgboostBinary26April.model')  # load data


In [ ]:
dfTest = pd.read_csv('../data/test.csv', delimiter=",", usecols = ['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price',  'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv'])
xgmat_test_submission = xgboost.DMatrix( dfTest)

In [ ]:
y_pred_clicking = bst.predict(xgmat_test_submission)


In [ ]:
bst = xgboost.Booster({'nthread': 12})  # init model
bst.load_model('bookingModel.model')  # load data

In [ ]:
y_pred_booking = bst.predict(xgmat_test_submission)

In [ ]:
y_pred = y_pred_clicking + y_pred_booking
dfTest['predictedPos'] = y_pred
dfTest = dfTest.sort_values(['srch_id','predictedPos'],ascending=[True, False])


In [ ]:
print(dfTest.head())
dfSubmission = dfTest[['srch_id','prop_id']]
dfSubmission.to_csv("submissionTest2.csv",index=False)

In [ ]:
#Counter(y_pred)
predictions = [round(value) for value in y_pred]
Counter(predictions)